# Mount Google Drive to access files

In [0]:
#!apt-get install tree
#!tree --help
#!tree --du -h gdrive/My\ Drive/Masterarbeit
!du -h --max-depth=1 gdrive/My\ Drive/

2.9M	gdrive/My Drive/Colab Notebooks
31G	gdrive/My Drive/Masterarbeit
903K	gdrive/My Drive/Hyve
31G	gdrive/My Drive/


---
# Imports & Config  

In [0]:

import os

class FLAGS_IC:
    # directories & files for tmp storage
    base_dir = "/content/gdrive/My Drive/Masterarbeit/ImageClassification"
    image_dir = base_dir + "/ImagePatches"


class FLAGS_OD:
    base_dir = "/content/gdrive/My Drive/Masterarbeit/ObjectDetection"
    
    image_dir = os.path.join(base_dir, "Images")
  

class FLAGS_TB:
    # a temporary folder to run tensorboard on, doesn't seem to run on a Gdrive mounted folder
    tmp_graph_dir = "/content/summaries"

    # how many images fit horizontally in an image mosaic
    tb_mosaic_width = 5

## Imports

## Connect to GPU

In [0]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Mount GDrive

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Pretty Print

In [0]:
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=2)
def print(kwargs):
  pp.pprint(kwargs)
  

# All Purpose Functions

In [0]:
import os
import tensorflow as tf

def get_xml_files(root_dir, selected_folders=None):
  """Gets all xml files recursively inside the root directory
  and returns the file names with the path
  Args:
    root_dir: root directory as string
    selected_folders: list of folder names which should only be
        used for the dataset. If None will use all.
  Returns:
    list of tuples with file_name and path_name relative from
        root_dir
  """
  file_list = []
  
  root_f = root_dir.split(os.sep)
  for path, _, files in tf.gfile.Walk(root_dir):
    path_f = path.split(os.sep)
    if not selected_folders is None and not \
        any(s_folder in path_f for s_folder in selected_folders):
      continue
    for file in files:
      if file.split(".")[-1] == "xml":
        rel_path = [f for f in path_f if f not in root_f]
        rel_path = os.sep.join(rel_path)
        file_list.append((file, rel_path))
  
  return file_list
  

In [0]:
import os
from bs4 import BeautifulSoup

def load_annotation(file_path):
  """
  Load annotation file for a given image and parse xml to dict.
  Args:
    img_name: string of the image name.
    infos: dict with information that should be appended to image dict
  Returns:
    dict of the image with all infos
  """
  xml = ""
  with open(file_path, 'r') as f:
    xml = f.readlines()
  xml = ''.join([line.strip('\t') for line in xml])
  bs = BeautifulSoup(xml)
  image_filename = os.path.join(*(file_path.split(os.sep)[:-1]))
  filename = bs.filename.string.replace("\n","").replace("\t","").replace(" ","")
  image_filename = os.path.join(image_filename, filename)
  image_filename = image_filename.replace("content/","")
  image = {"filename": image_filename.replace("\n",""),
           "size": {"width": int(bs.size.width.string),
                    "height": int(bs.size.height.string)},
           "objects": []}

  for ob in bs.find_all("object"):
    name = ob.find('name').string.replace("\n","").replace("\t","").replace(" ","")
          
    obj = {"name": name,
           "bndbox":{"xmin": int(ob.bndbox.xmin.string),
                     "xmax": int(ob.bndbox.xmax.string),
                     "ymin": int(ob.bndbox.ymin.string),
                     "ymax": int(ob.bndbox.ymax.string)},
           "pose":ob.pose.string,
           "truncated": bool(ob.truncated.string),
           "difficult": bool(ob.difficult.string)}
    image["objects"].append(obj)
  return image

In [0]:
def rotate_box(obj, height_old):
  xmin_tmp = height_old - obj["bndbox"]["ymax"]
  xmax_tmp = height_old - obj["bndbox"]["ymin"]
  ymin_tmp = obj["bndbox"]["xmin"]
  ymax_tmp = obj["bndbox"]["xmax"]
  obj["bndbox"]["xmin"] = xmin_tmp
  obj["bndbox"]["xmax"] = xmax_tmp
  obj["bndbox"]["ymin"] = ymin_tmp
  obj["bndbox"]["ymax"] = ymax_tmp


In [0]:
import cv2

def rotate_img(image):
  img = cv2.imread(image["filename"], 1)
  M = cv2.getRotationMatrix2D((image["size"]["height"]/2,image["size"]["width"]/2), 270, 1)
  rotated = cv2.warpAffine(img, M, (image["size"]["height"] ,image["size"]["width"]))
  cv2.imwrite(image["filename"], img)
  
  width_tmp = image["size"]["height"]
  height_tmp = image["size"]["width"]
  image["size"]["width"] = width_tmp
  image["size"]["height"] = height_tmp

In [0]:
import os

def append_basic(xml_list, image):
  xml_list.append("<folder>{}</folder>".format(image["filename"].split(os.sep)[-2]))
  xml_list.append("<filename>{}</filename>".format(image["filename"].split(os.sep)[-1]))
  xml_list.append("<path>{}</path>".format(image["filename"]))
  xml_list.append("<source>\n\t<database>Unknown</database>\n</source>")
  xml_list.append("<size>")
  xml_list.append("\t<width>{}</width>".format(image["size"]["width"]))
  xml_list.append("\t<height>{}</height>".format(image["size"]["height"]))
  xml_list.append("\t<depth>{}</depth>".format(3))
  xml_list.append("</size>")
  xml_list.append("<segmented>0</segmented>")

In [0]:
def append_obj(xml_list, obj):
  xml_list.append("<object>")
  xml_list.append("\t<name>{}</name>".format(obj["name"]))
  xml_list.append("\t<pose>Unspecified</pose>")
  xml_list.append("\t<truncated>0</truncated>")
  xml_list.append("\t<difficult>0</difficult>")
  xml_list.append("\t<bndbox>")
  xml_list.append("\t\t<xmin>{}</xmin>".format(obj["bndbox"]["xmin"]))
  xml_list.append("\t\t<ymin>{}</ymin>".format(obj["bndbox"]["ymin"]))
  xml_list.append("\t\t<xmax>{}</xmax>".format(obj["bndbox"]["xmax"]))
  xml_list.append("\t\t<ymax>{}</ymax>".format(obj["bndbox"]["ymax"]))
  xml_list.append("\t</bndbox>")
  xml_list.append("</object>")

In [0]:
from PIL import Image

def extract_patches(image):
  im=Image.open(image["filename"])
  
  for obj in image["objects"]:
    im=Image.open(image["filename"])

    bndbox = obj["bndbox"]
    bndbox = (bndbox["xmin"], bndbox["ymin"], bndbox["xmax"], bndbox["ymax"])

    im2=im.crop(bndbox)

    im2.save(".".join(image["filename"].split(".")[:-1]) + "_" + obj["name"] + ".png")

# Replace Labels

In [0]:
repl = {"can": "dose",
        "bottle": "flasche",
        "-": "_",
        "ö": "oe",
        "ä": "ae",
        "ü": "ue",}
repl_cls = {
            "coke___50__54491472": "coke__flasche_50__54491472",
}


files = get_xml_files(FLAGS_OD.image_dir, selected_folders=["_Automat"])
print(FLAGS_OD.image_dir)
print(len(files))
for file, path in files:
  print("working on:", file)
  xml_file = os.path.join(FLAGS_OD.image_dir, path, file)
  image = load_annotation(xml_file)
  xml_list = ["<annotation>"]
  append_basic(xml_list, image)
  
  for obj in image["objects"]:
    for key, val in repl.items():
      obj["name"] = obj["name"].replace(key,val)
    for key, val in repl_cls.items():
      if key in obj["name"]:
        obj["name"] = val
    append_obj(xml_list, obj)
    
  xml_list.append("</annotation>")    
  with open(xml_file, "w+") as file:
    file.write("\n".join(xml_list))  

/content/gdrive/My Drive/Masterarbeit/ObjectDetection/Images
295
working on: DSC01628.xml
working on: DSC01630.xml
working on: DSC01631.xml
working on: DSC01629.xml
working on: IMG_20181218_173039.xml
working on: DSC01646.xml
working on: IMG_20181218_175507.xml
working on: IMG_20181218_170655.xml
working on: DSC01627.xml
working on: DSC01633.xml
working on: IMG_20181218_171607.xml
working on: DSC01655.xml
working on: IMG_20181218_172857.xml
working on: DSC01667.xml
working on: DSC01651.xml
working on: IMG_20181218_170243.xml
working on: IMG_20181218_175503.xml
working on: DSC01635.xml
working on: IMG_20181218_171805.xml
working on: DSC01671.xml
working on: DSC01669.xml
working on: DSC01638.xml
working on: DSC01672.xml
working on: IMG_20181218_174605.xml
working on: IMG_20181218_170459.xml
working on: DSC01675.xml
working on: IMG_20181218_172432.xml
working on: IMG_20181218_170648.xml
working on: DSC01678.xml
working on: DSC01642.xml
working on: DSC01679.xml
working on: 20190111_115057_

# per image patch cleaning

In [0]:
from PIL import Image
from IPython.display import display, clear_output
import ipywidgets as widgets
import time
import os
import tensorflow as tf
from bs4 import BeautifulSoup

drinks = [
  "3068320353500",
  "76404160",
  "7610335002575"
  "9120025930135",
  "54491472",
  "5449238",
  "5449000131836",
  "40822938",
  "5449000235947",
  "5449000236623",
  "54490840",
  "7610057001078",
  "7610057012456",
  "90162909",
  "90162800",
  "7610097112079",
  "7610097111072",
  "7610335001530",
  "305764335648"
]

button = widgets.Button(description="Next")
radio = widgets.RadioButtons(
    options=['front', 'back', 'side'],
    description='pose:',
    disabled=False
)

cnt_out = widgets.Output()
img_out = widgets.Output()
display(cnt_out)
display(img_out)
display(radio)
display(button)


def load_annotation(file_path):
    """
    Load annotation file for a given image.
    Args:
        img_name (string): string of the image name, relative to
            the image directory.
    Returns:
        BeautifulSoup structure: the annotation labels loaded as a
            BeautifulSoup data structure
    """
    xml = ""
    with open(file_path) as f:
        xml = f.readlines()
    xml = ''.join([line.strip('\t') for line in xml])
    return BeautifulSoup(xml, "lxml")

def get_xml_files(root_dir, selected_folders=None):
    """Gets all xml files recursively inside the root directory
    and returns the file names with the path
    Args:
    root_dir: root directory as string
    selected_folders: list of folder names which should only be
        used for the dataset. If None will use all.
    Returns:
    list of tuples with file_name and path_name relative from
        root_dir
    """
    file_list = []

    root_f = root_dir.split(os.sep)
    for path, _, files in tf.gfile.Walk(root_dir):
        path_f = path.split(os.sep)
        if not selected_folders is None and not any(s_folder in path_f for s_folder in selected_folders):
            continue
        for file in files:
            if file.split(".")[-1] == "xml":
                rel_path = [f for f in path_f if f not in root_f]
                rel_path = os.path.join(*rel_path)
                file_list.append((file, rel_path))

    return file_list


def image_iter():
    image_dir = os.path.join(os.getcwd(),FLAGS_OD.image_dir)
    xmlfiles = get_xml_files(image_dir)
    it = 0
    for file_name, rel_path in xmlfiles:
        with cnt_out:
            clear_output()
            print(it, file_name)
        filename = os.path.join(image_dir, rel_path, file_name)
        pv = load_annotation(filename)
        img_filename = pv.annotation.filename.string.replace(" ","").replace("\n","")
        pil_im = Image.open(os.path.join(image_dir, rel_path, img_filename))
        for ob in pv.find_all("object"):
            gtin = ob.find('name').string.split("_")[-1]
            if not ob.pose.string in ["front", "back", "side"]:
                if gtin in drinks or "Freespace" in file_name:
                    crop_rectangle = (int(ob.bndbox.xmin.string), int(ob.bndbox.ymin.string), int(ob.bndbox.xmax.string), int(ob.bndbox.ymax.string))
                    pil_patch = pil_im.crop(crop_rectangle)
                    width, height = pil_patch.size
                    ref_size = 400
                    if width > height:
                        height = int(ref_size * height/width)
                        width = ref_size
                    else:
                        width = int(ref_size * width/height)
                        height = ref_size
                    size = (width, height)
                    pil_patch = pil_patch.resize(size)
                    with img_out:
                        clear_output()
                        display(pil_patch)
                    yield True
                    ob.pose.string = radio.value
                else:
                    ob.pose.string = "front"

        with open(filename, "+w") as f:
            f.write(str(pv.html.body.annotation))
            
        it += 1

img_iter = image_iter()
                    
def on_button_clicked(_):
    next(img_iter)
            
button.on_click(on_button_clicked)
        

Output()

Output()

RadioButtons(description='pose:', options=('front', 'back', 'side'), value='front')

Button(description='Next', style=ButtonStyle())

# Rotate Image

In [12]:
import os
%matplotlib inline
from matplotlib.pyplot import imshow
import cv2


base_dir = "/content/gdrive/My Drive/Masterarbeit"
object_detection_dir = os.path.join(base_dir, "ObjectDetection")

class FLAGS_OD:
  image_dir = os.path.join(object_detection_dir, "Images")

folder = "gdrive/My Drive/Masterarbeit/ObjectDetection/predict_folder/cleaning"
files = get_xml_files(folder)
     
print(len(files))
for file, path in files:
  print("working on: {}".format(file))
  xml_file = os.path.join(folder,file)
  image = load_annotation(xml_file)
  xml_list = ["<annotation>"]
  height_old = image["size"]["height"]
  rotate_img(image)
  print(image["size"])
  #append_basic(xml_list, image)
  #
  #for obj in image["objects"]:
  #  rotate_box(obj, height_old)
  #  append_obj(xml_list, obj)
  #  print(obj["bndbox"], obj["name"])
 # 
 # xml_list.append("</annotation>")
#
#  with open(xml_file, "w+") as file:
#    file.write("\n".join(xml_list))

1
working on: 20190125_114246.xml


error: ignored

# Move Files for Manual Cleaning

In [0]:
import shutil

name = "ramseier_jusdepomme__50__7610057001078"

dst_dir = "gdrive/My Drive/Masterarbeit/ObjectDetection/predict_folder/cleaning"

files = get_xml_files(FLAGS_OD.image_dir, selected_folders=["_Automat"])
for file, path in files:
  
  xml_file = os.path.join(FLAGS_OD.image_dir, path, file)
  image = load_annotation(xml_file)
  if any([name in obj["name"] for obj in image["objects"]]):
    print("copying: ", file)
    if not xml_file.split(os.sep)[-1] in os.listdir(dst_dir):
      shutil.copy(xml_file, os.path.join(dst_dir, xml_file.split(os.sep)[-1]))
    if not image["filename"].split(os.sep)[-1] in os.listdir(dst_dir):
      shutil.copy(image["filename"], os.path.join(dst_dir, image["filename"].split(os.sep)[-1]))